# Lumeo - Model Training

Train a U-Net model for low-light image enhancement on the LOL dataset.

**Model:** Heavy U-Net (~20M parameters)  
**Optimized for:** Free Colab tier with checkpointing  
**Training Time:** ~10-15 hours (across sessions)

## 1. Setup

In [ ]:
# Check GPU
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Install dependencies (if needed)
!pip install -q pytorch-msssim

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# UPDATE THIS PATH to your Drive location
DATASET_ROOT = '/content/drive/MyDrive/Lumeo/datasets'
CHECKPOINT_DIR = '/content/drive/MyDrive/Lumeo/checkpoints'

import os
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import vgg16, VGG16_Weights
from pytorch_msssim import ssim, SSIM
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
import time
from tqdm.auto import tqdm

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 2. Dataset

In [ ]:
class LOLDataset(Dataset):
    """LOL Dataset for low-light image enhancement"""
    
    def __init__(self, low_dir, high_dir, img_size=256, augment=True):
        self.low_dir = Path(low_dir)
        self.high_dir = Path(high_dir)
        self.img_size = img_size
        self.augment = augment
        
        # Get matched pairs
        low_files = set(f.name for f in self.low_dir.glob('*.png'))
        high_files = set(f.name for f in self.high_dir.glob('*.png'))
        self.image_names = sorted(list(low_files.intersection(high_files)))
        
        print(f"Found {len(self.image_names)} paired images")
        
        # Transforms
        self.to_tensor = transforms.ToTensor()
        self.resize = transforms.Resize((img_size, img_size), antialias=True)
    
    def __len__(self):
        return len(self.image_names)
    
    def __getitem__(self, idx):
        name = self.image_names[idx]
        
        # Load images
        low_img = Image.open(self.low_dir / name).convert('RGB')
        high_img = Image.open(self.high_dir / name).convert('RGB')
        
        # Convert to tensor [0, 1]
        low_tensor = self.to_tensor(low_img)
        high_tensor = self.to_tensor(high_img)
        
        # Resize
        low_tensor = self.resize(low_tensor)
        high_tensor = self.resize(high_tensor)
        
        # Augmentation
        if self.augment:
            # Random horizontal flip
            if torch.rand(1) > 0.5:
                low_tensor = torch.flip(low_tensor, [-1])
                high_tensor = torch.flip(high_tensor, [-1])
            
            # Random vertical flip
            if torch.rand(1) > 0.5:
                low_tensor = torch.flip(low_tensor, [-2])
                high_tensor = torch.flip(high_tensor, [-2])
        
        return low_tensor, high_tensor

In [ ]:
# Create datasets
TRAIN_LOW = os.path.join(DATASET_ROOT, 'LOLdataset/our485/low')
TRAIN_HIGH = os.path.join(DATASET_ROOT, 'LOLdataset/our485/high')
VAL_LOW = os.path.join(DATASET_ROOT, 'LOLdataset/eval15/low')
VAL_HIGH = os.path.join(DATASET_ROOT, 'LOLdataset/eval15/high')

train_dataset = LOLDataset(TRAIN_LOW, TRAIN_HIGH, img_size=256, augment=True)
val_dataset = LOLDataset(VAL_LOW, VAL_HIGH, img_size=256, augment=False)

# DataLoaders (num_workers=0 to avoid Colab multiprocessing issues)
BATCH_SIZE = 4  # Reduced for free tier GPU memory
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")

In [ ]:
# Visualize a batch
low_batch, high_batch = next(iter(train_loader))
print(f"Batch shape: {low_batch.shape}")

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
for i in range(4):
    axes[0, i].imshow(low_batch[i].permute(1, 2, 0).numpy())
    axes[0, i].set_title('Low-Light')
    axes[0, i].axis('off')
    
    axes[1, i].imshow(high_batch[i].permute(1, 2, 0).numpy())
    axes[1, i].set_title('Normal-Light')
    axes[1, i].axis('off')
plt.tight_layout()
plt.show()

## 3. U-Net Model (Heavy ~20M params)

In [ ]:
class ConvBlock(nn.Module):
    """Double convolution block with BatchNorm and ReLU"""
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        return self.conv(x)


class EncoderBlock(nn.Module):
    """Encoder block: ConvBlock + MaxPool"""
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = ConvBlock(in_ch, out_ch)
        self.pool = nn.MaxPool2d(2)
    
    def forward(self, x):
        skip = self.conv(x)
        down = self.pool(skip)
        return skip, down


class DecoderBlock(nn.Module):
    """Decoder block: Upsample + Concat + ConvBlock"""
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_ch, out_ch, 2, stride=2)
        self.conv = ConvBlock(in_ch, out_ch)
    
    def forward(self, x, skip):
        x = self.up(x)
        x = torch.cat([x, skip], dim=1)
        return self.conv(x)


class UNet(nn.Module):
    """Heavy U-Net for image enhancement (~20M params)"""
    def __init__(self, in_channels=3, out_channels=3):
        super().__init__()
        
        # Encoder (heavier channels)
        self.enc1 = EncoderBlock(in_channels, 64)
        self.enc2 = EncoderBlock(64, 128)
        self.enc3 = EncoderBlock(128, 256)
        self.enc4 = EncoderBlock(256, 512)
        
        # Bottleneck
        self.bottleneck = ConvBlock(512, 1024)
        
        # Decoder
        self.dec4 = DecoderBlock(1024, 512)
        self.dec3 = DecoderBlock(512, 256)
        self.dec2 = DecoderBlock(256, 128)
        self.dec1 = DecoderBlock(128, 64)
        
        # Output
        self.out_conv = nn.Conv2d(64, out_channels, 1)
        
        # Initialize weights
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        # Encoder
        skip1, x = self.enc1(x)
        skip2, x = self.enc2(x)
        skip3, x = self.enc3(x)
        skip4, x = self.enc4(x)
        
        # Bottleneck
        x = self.bottleneck(x)
        
        # Decoder
        x = self.dec4(x, skip4)
        x = self.dec3(x, skip3)
        x = self.dec2(x, skip2)
        x = self.dec1(x, skip1)
        
        # Output with sigmoid for [0, 1] range
        return torch.sigmoid(self.out_conv(x))

In [ ]:
# Create model
model = UNet().to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Model size: ~{total_params * 4 / 1e6:.1f} MB")

In [ ]:
# Test forward pass
test_input = torch.randn(1, 3, 256, 256).to(device)
test_output = model(test_input)
print(f"Input shape: {test_input.shape}")
print(f"Output shape: {test_output.shape}")
print(f"Output range: [{test_output.min():.3f}, {test_output.max():.3f}]")

## 4. Loss Functions

In [ ]:
class PerceptualLoss(nn.Module):
    """Perceptual loss using VGG16 features"""
    def __init__(self):
        super().__init__()
        vgg = vgg16(weights=VGG16_Weights.IMAGENET1K_V1).features[:16].eval()
        for param in vgg.parameters():
            param.requires_grad = False
        self.vgg = vgg
        self.criterion = nn.L1Loss()
        
        # ImageNet normalization
        self.register_buffer('mean', torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1))
        self.register_buffer('std', torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1))
    
    def normalize(self, x):
        return (x - self.mean) / self.std
    
    def forward(self, pred, target):
        pred_features = self.vgg(self.normalize(pred))
        target_features = self.vgg(self.normalize(target))
        return self.criterion(pred_features, target_features)


class CombinedLoss(nn.Module):
    """Combined loss: L1 + Perceptual + SSIM"""
    def __init__(self, l1_weight=1.0, perceptual_weight=0.1, ssim_weight=0.1):
        super().__init__()
        self.l1 = nn.L1Loss()
        self.perceptual = PerceptualLoss()
        self.ssim_loss = SSIM(data_range=1.0, size_average=True, channel=3)
        
        self.l1_weight = l1_weight
        self.perceptual_weight = perceptual_weight
        self.ssim_weight = ssim_weight
    
    def forward(self, pred, target):
        l1_loss = self.l1(pred, target)
        perc_loss = self.perceptual(pred, target)
        ssim_val = self.ssim_loss(pred, target)
        ssim_loss = 1 - ssim_val  # Convert to loss
        
        total = (self.l1_weight * l1_loss + 
                 self.perceptual_weight * perc_loss + 
                 self.ssim_weight * ssim_loss)
        
        return total, {
            'l1': l1_loss.item(),
            'perceptual': perc_loss.item(),
            'ssim': ssim_val.item()
        }

In [ ]:
# Initialize loss and optimizer
criterion = CombinedLoss().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200, eta_min=1e-6)

## 5. Training Loop with Checkpointing

In [ ]:
def compute_psnr(pred, target):
    """Compute PSNR in dB"""
    mse = torch.mean((pred - target) ** 2)
    if mse == 0:
        return float('inf')
    return 10 * torch.log10(1.0 / mse).item()


def save_checkpoint(model, optimizer, scheduler, epoch, best_psnr, path):
    """Save training checkpoint to Drive"""
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_psnr': best_psnr
    }, path)
    print(f"Checkpoint saved: {path}")


def load_checkpoint(path, model, optimizer=None, scheduler=None):
    """Load checkpoint from Drive"""
    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    return checkpoint['epoch'], checkpoint['best_psnr']

In [ ]:
def train_epoch(model, loader, criterion, optimizer):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    metrics = {'l1': 0, 'perceptual': 0, 'ssim': 0}
    
    pbar = tqdm(loader, desc='Training')
    for low, high in pbar:
        low, high = low.to(device), high.to(device)
        
        optimizer.zero_grad()
        output = model(low)
        loss, batch_metrics = criterion(output, high)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        for k in metrics:
            metrics[k] += batch_metrics[k]
        
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    n = len(loader)
    return total_loss / n, {k: v / n for k, v in metrics.items()}


@torch.no_grad()
def validate(model, loader, criterion):
    """Validate model"""
    model.eval()
    total_loss = 0
    total_psnr = 0
    total_ssim = 0
    
    for low, high in loader:
        low, high = low.to(device), high.to(device)
        output = model(low)
        loss, metrics = criterion(output, high)
        
        total_loss += loss.item()
        total_psnr += compute_psnr(output, high)
        total_ssim += metrics['ssim']
    
    n = len(loader)
    return total_loss / n, total_psnr / n, total_ssim / n

In [ ]:
# Training configuration
NUM_EPOCHS = 200
SAVE_EVERY = 10  # Save checkpoint every N epochs
RESUME = False   # Set True to resume from checkpoint

# Paths
checkpoint_path = os.path.join(CHECKPOINT_DIR, 'latest_checkpoint.pth')
best_model_path = os.path.join(CHECKPOINT_DIR, 'best_model.pth')

# Resume if needed
start_epoch = 0
best_psnr = 0

if RESUME and os.path.exists(checkpoint_path):
    start_epoch, best_psnr = load_checkpoint(checkpoint_path, model, optimizer, scheduler)
    start_epoch += 1
    print(f"Resuming from epoch {start_epoch}, best PSNR: {best_psnr:.2f}")

In [ ]:
# Training loop
history = {'train_loss': [], 'val_loss': [], 'psnr': [], 'ssim': []}

print(f"Starting training from epoch {start_epoch}...")
print(f"Training on {len(train_dataset)} images, validating on {len(val_dataset)} images")
print("="*60)

for epoch in range(start_epoch, NUM_EPOCHS):
    start_time = time.time()
    
    # Train
    train_loss, train_metrics = train_epoch(model, train_loader, criterion, optimizer)
    
    # Validate
    val_loss, val_psnr, val_ssim = validate(model, val_loader, criterion)
    
    # Update scheduler
    scheduler.step()
    
    # Log
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['psnr'].append(val_psnr)
    history['ssim'].append(val_ssim)
    
    epoch_time = time.time() - start_time
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} ({epoch_time:.1f}s) | "
          f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
          f"PSNR: {val_psnr:.2f} dB | SSIM: {val_ssim:.4f}")
    
    # Save best model
    if val_psnr > best_psnr:
        best_psnr = val_psnr
        torch.save(model.state_dict(), best_model_path)
        print(f"  --> New best model saved! PSNR: {best_psnr:.2f}")
    
    # Save checkpoint for resuming
    if (epoch + 1) % SAVE_EVERY == 0:
        save_checkpoint(model, optimizer, scheduler, epoch, best_psnr, checkpoint_path)

print("="*60)
print(f"Training complete! Best PSNR: {best_psnr:.2f} dB")

## 6. Training Visualization

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(history['train_loss'], label='Train')
axes[0].plot(history['val_loss'], label='Val')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Loss')
axes[0].legend()

axes[1].plot(history['psnr'])
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('PSNR (dB)')
axes[1].set_title('Validation PSNR')

axes[2].plot(history['ssim'])
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('SSIM')
axes[2].set_title('Validation SSIM')

plt.tight_layout()
plt.savefig(os.path.join(CHECKPOINT_DIR, 'training_curves.png'), dpi=150)
plt.show()

## 7. Qualitative Results

In [ ]:
# Load best model
model.load_state_dict(torch.load(best_model_path, map_location=device))
model.eval()

@torch.no_grad()
def enhance_image(model, img_tensor):
    """Enhance a single image"""
    return model(img_tensor.unsqueeze(0).to(device)).squeeze(0).cpu()

# Visualize results on validation set
fig, axes = plt.subplots(5, 3, figsize=(12, 20))

for idx in range(5):
    low, high = val_dataset[idx]
    enhanced = enhance_image(model, low)
    
    axes[idx, 0].imshow(low.permute(1, 2, 0).numpy())
    axes[idx, 0].set_title('Input (Low-Light)')
    axes[idx, 0].axis('off')
    
    axes[idx, 1].imshow(enhanced.permute(1, 2, 0).numpy())
    axes[idx, 1].set_title('Enhanced')
    axes[idx, 1].axis('off')
    
    axes[idx, 2].imshow(high.permute(1, 2, 0).numpy())
    axes[idx, 2].set_title('Ground Truth')
    axes[idx, 2].axis('off')

plt.tight_layout()
plt.savefig(os.path.join(CHECKPOINT_DIR, 'validation_results.png'), dpi=150)
plt.show()

## 8. Export Model for Backend

In [ ]:
# Save model weights only (for deployment)
export_path = os.path.join(CHECKPOINT_DIR, 'lumeo_unet.pth')
torch.save(model.state_dict(), export_path)
print(f"Model exported to: {export_path}")
print(f"Model size: {os.path.getsize(export_path) / 1e6:.1f} MB")

In [ ]:
print("\n" + "="*60)
print("TRAINING COMPLETE!")
print("="*60)
print(f"Best PSNR: {best_psnr:.2f} dB")
print(f"Model saved to: {export_path}")
print("\nNext steps:")
print("1. Download lumeo_unet.pth from Drive")
print("2. Use it in FastAPI backend for inference")